In [180]:
import requests as rq
import json
import pandas as pd
pd.options.plotting.backend = "plotly"
import numpy as np
import warnings
import pytz
from datetime import datetime
import time
from IPython.display import clear_output

In [2]:
# Get demo API key
def get_demo_key():
    f = open("/home/vikas/Documents/CG_demo_key.json")
    key_dict = json.load(f)
    return key_dict["key"] 

In [7]:
use_demo = {
           "accept": "application/json",
           "x-cg-demo-api-key" : get_demo_key() 
}

In [3]:
# Get pro API key
def get_pro_key():
    f = open("/home/vikas/Documents/CG_pro_key.json")
    key_dict = json.load(f)
    return key_dict["key"] 

## Get list of exchanges

In [4]:
PUB_URL = "https://api.coingecko.com/api/v3"
PRO_URL = "https://pro-api.coingecko.com/api/v3"

In [5]:
def get_response(endpoint, headers, params, URL):
    url = "".join((URL, endpoint))
    response = rq.get(url, headers = headers, params = params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data, check status code {response.status_code}")    

In [9]:
# Valid values for results per page is between 1-250
exchange_params = {
            "per_page": 250,
            "page": 1
}

In [10]:
exchange_list_response = get_response("/exchanges", use_demo, exchange_params, PUB_URL)

In [11]:
df_ex = pd.DataFrame(exchange_list_response)

In [12]:
df_ex

,id,name,year_established,country,description,url,image,has_trading_incentive,trust_score,trust_score_rank,trade_volume_24h_btc,trade_volume_24h_btc_normalized
0,bybit_spot,Bybit,2018.0,British Virgin Islands,Bybit is a cryptocurrency exchange that offers...,https://www.bybit.com,https://coin-images.coingecko.com/markets/imag...,False,10.0,1.0,32158.188622,22423.270927
1,gdax,Coinbase Exchange,2012.0,United States,,https://www.coinbase.com/,https://coin-images.coingecko.com/markets/imag...,False,10.0,2.0,14825.942967,14825.942967
2,okex,OKX,2017.0,Seychelles,,https://www.okx.com,https://coin-images.coingecko.com/markets/imag...,False,10.0,3.0,14619.888740,14619.888740
3,gate,Gate.io,NaN,Cayman Islands,"Gate was established in 2013, and it is the to...",https://www.gate.io/,https://coin-images.coingecko.com/markets/imag...,False,10.0,4.0,17907.743194,11861.251440
4,kucoin,KuCoin,2017.0,Seychelles,"Launched in September 2017, KuCoin is a global...",https://www.kucoin.com/,https://coin-images.coingecko.com/markets/imag...,False,10.0,5.0,5974.994369,4433.619730
...,...,...,...,...,...,...,...,...,...,...,...,...
245,curve-bsc,Curve (BSC),NaN,None,,https://curve.fi/#/bsc/swap,https://coin-images.coingecko.com/markets/imag...,False,NaN,244.0,2.417181,2.417181
246,bunnyswap,BunnySwap,NaN,None,,,https://coin-images.coingecko.com/markets/imag...,False,5.0,245.0,1.990213,1.990213
247,spartadex,SpartaDEX,2023.0,Estonia,,https://app.spartadex.io/dex,https://coin-images.coingecko.com/markets/imag...,False,5.0,246.0,2.067631,2.067631
248,pearl-exchange,PearlFi,NaN,None,,https://www.pearl.exchange/swap,https://coin-images.coingecko.com/markets/imag...,False,5.0,247.0,1.932746,1.932746


In [13]:
list(df_ex.columns)

['id',
 'name',
 'year_established',
 'country',
 'description',
 'url',
 'image',
 'has_trading_incentive',
 'trust_score',
 'trust_score_rank',
 'trade_volume_24h_btc',
 'trade_volume_24h_btc_normalized']

#### List exchanges by trading volume

In [58]:
df_ex_subset = df_ex[["id", "name", "country", "trade_volume_24h_btc"]]
df_ex_subset = df_ex_subset.sort_values(by = ["trade_volume_24h_btc"], ascending = False)

In [66]:
df_ex_subset["country"].unique()

array(['Cayman Islands', 'Seychelles', 'British Virgin Islands',
       'United States', 'Lithuania', 'Singapore', 'Malta', 'South Korea',
       'Canada', None, 'United Arab Emirates', 'Gibraltar', 'Australia',
       'Bermuda', 'Hong Kong', 'Spain', 'Turkey', 'Netherlands',
       'Bahamas', 'Luxembourg', 'Saint Vincent and the Grenadines',
       'United Kingdom', 'Vanuatu', 'Belize', 'Argentina', 'Costa Rica',
       'Samoa', 'Estonia', 'Thailand', 'Japan', 'Indonesia', 'Taiwan',
       'Philippines', 'Switzerland', 'Brazil', 'India', 'South Africa',
       'Unknown or Invalid Region', 'Liechtenstein', 'Panama', 'Georgia',
       'Jamaica', 'Slovenia'], dtype=object)

In [72]:
df_ex_subset[(df_ex_subset["country"] == "Hong Kong")]

,id,name,country,trade_volume_24h_btc
12,hashkey_exchange,HashKey Exchange,Hong Kong,1830.439435
113,tokpie,Tokpie,Hong Kong,75.880736


## Get exchange tickers

In [78]:
def get_curr_exchange(id, base_curr, target_curr):
    
    exchange_ticker_response = get_response(f"/exchanges/{id}/tickers",
                                            use_demo,
                                            {},
                                            PUB_URL)
    
    found_match = ""
    
    for ticker in exchange_ticker_response["tickers"]:
        if ticker["base"] == base_curr and ticker["target"] == target_curr:
            found_match = ticker
            break
            
    if found_match == "":
        warnings.warn(f"No data found for {base_curr}-{target_curr} pair in {id}")
    
    return found_match

In [127]:
test_data = get_curr_exchange("gdax", "BTC", "USD")

In [128]:
test_data

{'base': 'BTC',
 'target': 'USD',
 'market': {'name': 'Coinbase Exchange',
  'identifier': 'gdax',
  'has_trading_incentive': False},
 'last': 64304.83,
 'volume': 2418.27443904,
 'converted_last': {'btc': 1.000403, 'eth': 18.370559, 'usd': 64305},
 'converted_volume': {'btc': 2419, 'eth': 44425, 'usd': 155506727},
 'trust_score': 'green',
 'bid_ask_spread_percentage': 0.010016,
 'timestamp': '2024-06-22T21:58:01+00:00',
 'last_traded_at': '2024-06-22T21:58:01+00:00',
 'last_fetch_at': '2024-06-22T21:59:00+00:00',
 'is_anomaly': False,
 'is_stale': False,
 'trade_url': 'https://www.coinbase.com/advanced-trade/spot/BTC-USD',
 'token_info_url': None,
 'coin_id': 'bitcoin'}

## Convert to local timezone

In [107]:
"UTC" in pytz.all_timezones

True

In [118]:
def convert_to_local_tz(old_ts):
    
    new_tz = pytz.timezone("Europe/Amsterdam")
    old_tz = pytz.timezone("UTC")
    
    format = "%Y-%m-%dT%H:%M:%S+00:00"
    datetime_obj = datetime.strptime(old_ts, format)
    
    localized_ts = old_tz.localize(datetime_obj)
    new_ts = localized_ts.astimezone(new_tz)
    
    return new_ts   

## Collect data for multiple exchanges based on country

In [131]:
def get_curr_all_exchanges(country,
                           base_curr,
                           target_curr):
    
    df_all = df_ex_subset[(df_ex_subset["country"] == country)]    
    
    exchanges_list = df_all["id"]
    ex_all = []    
       
    for exchange_id in exchanges_list:
        found_match = get_curr_exchange(exchange_id, base_curr, target_curr)
        if found_match == "":
            continue
        else:
            temp_dict = dict(
                             exchange = exchange_id,
                             last_price = found_match["last"],
                             last_vol   = found_match["volume"],
                             spread     = found_match["bid_ask_spread_percentage"],
                             trade_time = convert_to_local_tz(found_match["last_traded_at"])
                             )
            ex_all.append(temp_dict)
            
    return pd.DataFrame(ex_all)   

In [194]:
df_ex_all = get_curr_all_exchanges("United States", "BTC", "USDT")
df_ex_all

/tmp/ipykernel_25321/2761537337.py:16: UserWarning: No data found for BTC-USDT pair in itbit
  warnings.warn(f"No data found for {base_curr}-{target_curr} pair in {id}")
/tmp/ipykernel_25321/2761537337.py:16: UserWarning: No data found for BTC-USDT pair in metalx
  warnings.warn(f"No data found for {base_curr}-{target_curr} pair in {id}")


,exchange,last_price,last_vol,spread,trade_time
0,gdax,61042.53,1368.505541,0.019417,2024-06-29 14:48:02+02:00
1,tapbit,61051.97,6682.208800,0.010016,2024-06-29 14:48:33+02:00
2,kraken,61058.70,74.463866,0.010164,2024-06-29 14:49:02+02:00
3,gemini,61132.66,1.616248,0.023160,2024-06-29 14:18:02+02:00
4,binance_us,61058.19,22.039947,0.060537,2024-06-29 14:51:06+02:00
5,coinlist,61045.83,0.824200,0.050453,2024-06-29 14:49:03+02:00


## Collect and display data over a period of time

In [200]:
def display_agg_per_exchange(df_ex_all):
    
    # Group data and calculate statistics per exchange    
    df_agg = (
        df_ex_all.groupby("exchange").agg
        (        
            trade_time_min = ("trade_time", 'min'),
            trade_time_latest = ("trade_time", 'max'),
            last_price_mean = ("last_price", 'mean'),
            last_vol_mean = ("last_vol", 'mean'),
            spread_mean = ("spread", 'mean'),
            num_trades = ("last_price", 'count')
        )
    )
    
    # Get time interval over which statistics have been calculated    
    df_agg["trade_time_duration"] = df_agg["trade_time_latest"] - df_agg["trade_time_min"]
    
    # Remove redundant column
    df_agg = df_agg.drop(columns = ["trade_time_min"])
    
    with pd.option_context('display.precision', 2,
                           'display.colheader_justify', 'center'):
        
        display(df_agg.style.apply(highlight_max_min,
                                   color = 'green',
                                   subset = "last_price_mean")
               )
    
    return None    

In [213]:
def highlight_max_min(x, color):
    
    return np.where((x == np.nanmax(x.to_numpy())) |
                    (x == np.nanmin(x.to_numpy())),
                    f"color: {color};",
                    None)

In [214]:
display_agg_per_exchange(df_ex_all)

,trade_time_latest,last_price_mean,last_vol_mean,spread_mean,num_trades,trade_time_duration
exchange,,,,,,
binance_us,2024-06-29 14:51:06+02:00,61058.190000,22.039947,0.060537,1,0 days 00:00:00
coinlist,2024-06-29 14:49:03+02:00,61045.830000,0.824200,0.050453,1,0 days 00:00:00
gdax,2024-06-29 14:48:02+02:00,61042.530000,1368.505541,0.019417,1,0 days 00:00:00
gemini,2024-06-29 14:18:02+02:00,61132.660000,1.616248,0.023160,1,0 days 00:00:00
kraken,2024-06-29 14:49:02+02:00,61058.700000,74.463866,0.010164,1,0 days 00:00:00
tapbit,2024-06-29 14:48:33+02:00,61051.970000,6682.208800,0.010016,1,0 days 00:00:00


In [188]:
def run_data_bot(country,
                 base_curr,
                 target_curr):
    
    df_ex_all = get_curr_all_exchanges(country, base_curr, target_curr)
    
    # Collect data every minute    
    while True:
        time.sleep(60)
        df_new = get_curr_all_exchanges(country, base_curr, target_curr)
        
        # Merge to existing DataFrame
        df_ex_all = pd.concat([df_ex_all, df_new])
        
        # Remove duplicate rows based on all columns
        df_ex_all = df_ex_all.drop_duplicates()
        
        # Clear previous display once new one is available
        clear_output(wait = True)
        display_agg_per_exchange(df_ex_all)        
        
    return None        

In [193]:
#run_data_bot("United States", "BTC", "USDT")